In [2]:
import numpy as np
import igl
import meshplot as mp
import math
import scipy.sparse as sp
from numpy import matlib as mb
from math import sqrt

# Load Data

In [247]:
v, f = igl.read_triangle_mesh('../data/camel_head.off')

kdmin, kdmax, kmin, kmax = igl.principal_curvature(v,f)  
kdmax /= np.linalg.norm(kdmax)
kdmin /= np.linalg.norm(kdmin)
# mp.plot(v, f, c=kmin)
# mp.plot(v, f, c=kmax)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

In [194]:
def getAdjacentFaces(v,f):
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    VFi = []
    for i in range(NI.shape[0] - 1):
        VFii = []
        jj = NI[i + 1] - NI[i]
        for j in range(jj):
            VFii.append(VF[NI[i] + j])
        VFi.append(VFii)
    return VFi

# Find Regular Triangle

In [248]:
def sgn(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

def mark_regular(f,kdmax,kdmin):
    singular = []
    regular = []
    color = []
    for indfi, fi in enumerate(f):
        sign_min = 1
        sign_max = 1
        for i in range(1,3):
            if np.dot(kdmax[fi[0]],kdmax[fi[i]]) < 0:
                kdmax[fi[i]] = - kdmax[fi[i]]
            if np.dot(kdmin[fi[0]],kdmin[fi[i]]) < 0:
                kdmin[fi[i]] = - kdmin[fi[i]]
        if np.dot(kdmax[fi[1]],kdmax[fi[2]]) > 0 and np.dot(kdmin[fi[1]],kdmin[fi[2]]) >0:
            regular.append(indfi)
            color.append([1,1,1])
        else:
            singular.append(indfi)
            color.append([0.8,0.8,0.8])
    return np.array(regular), np.array(singular), np.array(color)

regular, singular,color = mark_regular(f,kdmax,kdmin)

# Compute Extremalities

In [249]:
def extremalities(v, f, k, kd):
    EX = np.zeros((v.shape[0],1))
    A = igl.doublearea(v,f)
    G = igl.grad(v, f)
    grad_k = G * k
    neighbour = getAdjacentFaces(v,f) 
    for indvi, vi in enumerate(v):
        area = 0
        ex = 0
        # vfi: T, indvi: p
        for indvfi, vfi in enumerate(neighbour[indvi]):
            area += A[vfi] # compute area star
        for indvfi, vfi in enumerate(neighbour[indvi]):
            g = np.array([grad_k[vfi], grad_k[f.shape[0] + vfi], grad_k[2 * f.shape[0] + vfi]])
            ex += A[vfi] * np.dot(g,kd[indvi])
        ex /= area
        EX[indvi] = ex
    return EX

EX_max = extremalities(v, f, kmax, kdmax)
EX_min = extremalities(v, f, kmin, kdmin)

# Smooth Extremalities for 10 Rounds

In [207]:
def smooth_extremalities(v, f, k, kd, ex):
    A = igl.adjacency_list(f)
    L = igl.cotmatrix(v,f)
#     M = igl.massmatrix(v,f)
#     L = M.T * L 
    LEx = np.zeros((v.shape[0],1))
    for indvi, vi in enumerate(v):
        lp = 0
        for vn in A[indvi]:
            sign = sgn(np.dot(kd[indvi],kd[vn]))
            lp += L[indvi,vn] * (sign * ex[vn] - ex[indvi])
        LEx[indvi] = lp
    ex += 0.02 * LEx
    return ex
for i in range(10): # smooth for 10 rounds
    EX_max = smooth_extremalities(v, f, kmax, kdmax, EX_max)
    EX_min = smooth_extremalities(v, f, kmin, kdmin, EX_min)

# Extract Feature Line

In [250]:
def extract_feature_line(v, f, kmax, kmin, KD, EX, regular, singular, sign, T):
    # for regular faces
    fr = np.array([f[index] for index in regular])
    TT, TTi = igl.triangle_triangle_adjacency(fr) # id of the adjacent triangle, id of the shared edge
    marked_edges = []
    zero_point_v = []
    zero_point_k = []
    zero_edges_ind = [[],[]]
    zero_edges = [[],[]]
    
    # for regular triangles
    for indfri, fri in enumerate(fr):
        ex = np.array([EX[index] for index in fri]) # each vertex's extremalities on the current triangle
        kd = np.array([KD[index] for index in fri]) # each vertex's curvature direction on the current triangle
        # flip signs
        for i in range(1,3):
            if np.dot(kd[0],kd[i]) < 0:
                kd[i] = - kd[i]
                ex[i] = - ex[i]

        if ex[ex.argmax()] <= 0.0 or ex[ex.argmin()] >= 0.0:
            continue # no zero points
        kd_sum = kd.sum(axis=0)
        vv = np.array([v[index] for index in fri]) # the vertex on this triangle
        G = igl.grad(vv, np.array([[0,1,2]]))
        gex = G * ex # trangle based gradient
        
        # check equation (5)
        if sign * np.dot(np.array([kd_sum]),gex) >= 0: 
            continue
            
        # check equation (6)
        kmax_ = np.array([kmax[index] for index in fri])
        kmin_ = np.array([kmin[index] for index in fri])
        if sign * (abs(kmax_.sum(axis=0)) - abs(kmin_.sum(axis=0))) <= 0:
            continue
            
        count = 0
        for i in range(3):
            j = (i + 1) % 3
            if ex[i] * ex[j] <= 0: # has sign change: should have a zero point in the middle
                # add mid point
                a = abs(ex[i])
                b = abs(ex[j])
                mid = (b * v[fri[i]] + a * v[fri[j]]) / (a + b)
                
                if [mid[0],mid[1],mid[2]] not in zero_point_v:
                    zero_point_v.append([mid[0],mid[1],mid[2]])
                    ind = len(zero_point_v) -1
                    if sign == 1:
                        zero_point_k.append((b * kmax[fri[i]] + a * kmax[fri[j]]) / (a + b))
                    else:
                        zero_point_k.append((b * kmin[fri[i]] + a * kmin[fri[j]]) / (a + b))
                else:
                    ind = zero_point_v.index([mid[0],mid[1],mid[2]])
                zero_edges_ind[count].append(ind)
                zero_edges[count].append(mid)
                # mark edge i, j
                marked_edges.append([fri[i],fri[j]])
                count = (count + 1)%2
         
    # for singular triangles           
    for indfi, fi in enumerate(singular):
        face = f[fi]
        marked = []
        for es in range(3): # edge start 0-1, 1-2, 2-0
            ee = (es + 1) % 3 # edge end
            # see whether there are marked edges
            if [face[es],face[ee]] in marked_edges or [face[ee],face[es]] in marked_edges:
                marked.append([face[es],face[ee]])
        if len(marked) < 2:
            continue # one marked edge: do nothing
        if len(marked) == 2:
            for k in range(2):
                i = marked[k][0]
                j = marked[k][1]
                a = abs(EX[i])
                b = abs(EX[j])
                mid = (b * v[i] + a * v[j]) / (a + b)
                if [mid[0],mid[1],mid[2]] not in zero_point_v:
                    zero_point_v.append([mid[0],mid[1],mid[2]])
                    ind = len(zero_point_v) -1
                    if sign == 1:
                        zero_point_k.append((b * kmax[fri[i]] + a * kmax[fri[j]]) / (a + b))
                    else:
                        zero_point_k.append((b * kmin[fri[i]] + a * kmin[fri[j]]) / (a + b))
                else:
                    ind = zero_point_v.index([mid[0],mid[1],mid[2]])
                zero_edges_ind[k].append(ind)
                zero_edges[k].append(mid)
                
        if len(marked) == 3:
            v0 = v[face][0]
            v1 = v[face][1]
            v2 = v[face][2]
            bc = (v0 + v1 + v2)/3 # barycenter
            if sign == 1:
                k_temp = (kmax[face[0]] + kmax[face[1]] + kmax[face[2]]) / 3
            else:
                k_temp = (kmin[face[0]] + kmin[face[1]] + kmin[face[2]]) / 3
            
            for k in range(3):
                zero_point_k.append(k_temp)
                zero_edges_ind[1].append(ind)
                zero_edges[1].append(bc)
            for k in range(3):
                i = marked[k][0]
                j = marked[k][1]
                a = abs(EX[i])
                b = abs(EX[j])
                mid = (b * v[i] + a * v[j]) / (a + b)
                if [mid[0],mid[1],mid[2]] not in zero_point_v:
                    zero_point_v.append([mid[0],mid[1],mid[2]])
                    ind = len(zero_point_v) -1
                    if sign == 1:
                        zero_point_k.append((b * kmax[fri[i]] + a * kmax[fri[j]]) / (a + b))
                    else:
                        zero_point_k.append((b * kmin[fri[i]] + a * kmin[fri[j]]) / (a + b))
                else:
                    ind = zero_point_v.index([mid[0],mid[1],mid[2]])
                zero_edges_ind[1].append(ind)
                zero_edges[1].append(mid)
                

    # Remove small ridges by a threshold filter
    visited = [np.zeros(len(zero_edges_ind[0])),np.zeros(len(zero_edges_ind[1]))]
    lines = [[],[]]
    while 0 in visited[0] or 0 in visited[1]: # there are still unvisited edges
        temp_line = [[],[]]
        itg_k = 0 # the trapezoid approximation of the integral
        # pick a point to start
        zv_count = 0
        if 0 in visited[0]:
            ind_cur = np.where(visited[0]==0)[0][0]
            which_end = 0
        else:
            ind_cur = np.where(visited[1]==0)[0][0]
            which_end = 1
            
        notEnd = True
        temp_line[0].append(zero_edges[0][ind_cur]) 
        temp_line[1].append(zero_edges[1][ind_cur])
        visited[0][ind_cur] = 1
        visited[1][ind_cur] = 1
        ind_a = zero_edges_ind[0][ind_cur]
        ind_b = zero_edges_ind[1][ind_cur]
        itg_k += 0.5 * (zero_point_k[ind_a] + zero_point_k[ind_a])*np.linalg.norm(zero_edges[0][ind_cur] - zero_edges[1][ind_cur])
        # trace along the line
        while notEnd:
            if zero_edges_ind[(which_end + 1)%2][ind_cur] in zero_edges_ind[which_end]:
                ind_cur = zero_edges_ind[which_end].index(zero_edges_ind[(which_end + 1)%2][ind_cur])
                temp_line[0].append(zero_edges[0][ind_cur]) 
                temp_line[1].append(zero_edges[1][ind_cur]) 
                visited[0][ind_cur] = 1
                visited[1][ind_cur] = 1
                ind_a = zero_edges_ind[0][ind_cur]
                ind_b = zero_edges_ind[1][ind_cur]
                itg_k += 0.5 * (zero_point_k[ind_a] + zero_point_k[ind_a])*np.linalg.norm(zero_edges[0][ind_cur] - zero_edges[1][ind_cur])
            elif np.where(np.array(zero_edges_ind[(which_end + 1)%2])==zero_edges_ind[(which_end + 1)%2][ind_cur])[0].shape[0] > 1:
                for i in np.where(np.array(zero_edges_ind[(which_end + 1)%2])==zero_edges_ind[(which_end + 1)%2][ind_cur])[0]:
                    if i != ind_cur:
                        ind_cur = i
                        which_end = (which_end + 1)%2
                        temp_line[0].append(zero_edges[0][ind_cur]) 
                        temp_line[1].append(zero_edges[1][ind_cur]) 
                        visited[0][ind_cur] = 1
                        visited[1][ind_cur] = 1
                        ind_a = zero_edges_ind[0][ind_cur]
                        ind_b = zero_edges_ind[1][ind_cur]
                        itg_k += 0.5 * (zero_point_k[ind_a] + zero_point_k[ind_a])*np.linalg.norm(zero_edges[0][ind_cur] - zero_edges[1][ind_cur])
                        break

            else:
                notEnd = False
        if sign * itg_k > T:
            for indzv, zvi in enumerate(temp_line[0]):
                lines[0].append(zvi)
                lines[1].append(temp_line[1][indzv])

    return np.array(lines)

lines_max = extract_feature_line(v,f,kmax,kmin,kdmax,EX_max,regular, singular, 1, 0.8)
lines_min = extract_feature_line(v,f,kmax,kmin,kdmin,EX_min,regular, singular, -1, 0.8)

# Draw Feature Lines

In [251]:
p = mp.plot(v, f, c=color)
p.add_lines(lines_max[0], lines_max[1], shading={"line_color": "red"})
p.add_lines(lines_min[0], lines_min[1], shading={"line_color": "blue"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

2